# EvoGFuzz++

In [ ]:
import string
import math
from fuzzingbook.Grammars import Grammar

GRAMMAR: Grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"]
}

INITIAL_INPUTS = ['cos(10)', 'sqrt(28367)', 'tan(-12)', 'sqrt(3)']

Let's parse the initial input files an observe if they are valid according to the grammar

In [ ]:
from fuzzingbook.Parser import EarleyParser, tree_to_string, DerivationTree

p = EarleyParser(GRAMMAR)

for inp in INITIAL_INPUTS:
    for tree in p.parse(inp):
        assert tree_to_string(tree) == inp, f"{tree} != {inp}"

In [ ]:
def arith_eval(inp: DerivationTree) -> float:
    return eval(str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan})

# Returns true if a Error is found
def prop(inp: DerivationTree) -> bool:
    try:
        arith_eval(inp)
        return False
    except ValueError:
        return True


In [ ]:
print([(x, prop(x)) for x in INITIAL_INPUTS])

In [ ]:
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s:  %(message)s"
)

In [ ]:
from evogfuzz.evogfuzz_class import EvoGFuzz
from evogfuzz.fitness_functions import fitness_function_failure as fitness_function

epp = EvoGFuzz(GRAMMAR,
               prop,
               INITIAL_INPUTS,
               fitness_function=fitness_function,
               iterations=50,
               working_dir=None
               )

In [ ]:
epp.execute()

In [ ]:
display(epp._probabilistic_grammars)

In [ ]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer

prob_fuzzer = ProbabilisticGrammarFuzzer(epp._probabilistic_grammars[-1][0])

for _ in range(100):
    print(prob_fuzzer.fuzz())
